In [ ]:
import os
import re
import time
import datetime
from tqdm import tqdm 
import pandas as pd
import requests
import ast
from IPython.display import display

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.configs import BASE_URL, URLS_FRAGMENTS, COLUMN_NAME
from utils.helper_func import build_complete_urls, extract_company_name, combined_df, parse_admin_region, parse_sale_period, find_first_sale_time, sales_start_time

In [ ]:
urls = build_complete_urls(BASE_URL, URLS_FRAGMENTS)
print(urls)

In [ ]:
community_raw_df = combined_df(urls, "20250801")

In [ ]:
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\raw"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
csv_fn = "community_raw_data_20280801.csv"
out_path = os.path.join(output_dir,  csv_fn)
community_raw_df.to_csv(out_path, index=False, encoding='utf-8-sig')

In [ ]:
input_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\raw"
csv_fn = "community_raw_data_20280801.csv"
input_path = os.path.join(input_dir,  csv_fn)

df = pd.read_csv(input_path, encoding='utf-8-sig')

In [ ]:
df.columns

In [ ]:
df = df.rename(columns=COLUMN_NAME, inplace=False)
display(df.columns)
display(df.shape)

In [ ]:
pd.set_option('display.max_columns', None)  # 所有欄位都顯示
pd.set_option('display.max_rows', 500)      # 最多顯示 100 列
# pd.reset_option('display.max_colwidth', None) # 不截斷欄位內容


In [ ]:
proc_df = df.copy()
# 取出建商名稱，並建立company_name欄位
proc_df['建設公司'] = proc_df.apply(extract_company_name, axis=1)
# 在 proc_df 裡面新增一個「行政區」欄位
proc_df["行政區"] = proc_df["坐落街道"].apply(parse_admin_region)
# proc_df["自售期間"], proc_df["代銷期間"] = zip(*proc_df["銷售期間"].apply(parse_sale_period))

# 依規則從「自售期間」及「代銷期間」欄位提取出7位數字，
# 分別存入新欄位「自售起始時間」與「代銷起始時間」
proc_df["自售起始時間"] = proc_df["自銷售期間"].apply(find_first_sale_time)
proc_df["代銷起始時間"] = proc_df["代銷售期間"].apply(find_first_sale_time)


In [ ]:
# proc_df[['自售起始時間', '代銷起始時間', '自銷售期間', '代銷售期間']].sample(200)
# proc_df[(proc_df['自售起始時間'].isna() & proc_df['代銷起始時間'].isna())][['自銷售期間', '代銷售期間']]

In [ ]:
# 建立「銷售起始時間」欄位
proc_df["銷售起始時間"] = proc_df.apply(sales_start_time, axis=1)

In [ ]:
# proc_df[proc_df['銷售起始時間'].isna()]

In [ ]:
proc_df.columns

In [ ]:
community_df = proc_df[[ '編號', '縣市', '行政區', '社區名稱', '戶數',  '主要用途', '銷售起始時間', '完成建物第一次登記日期','建照執照',  
                        '自售起始時間', '代銷起始時間', '備查完成日期', '建照核發日', '自銷售期間', '代銷售期間', '經度','緯度', '主要建材','坐落街道','使用分區', '坐落基地', '匯入時間']]
community_df


In [ ]:
# 輸出的資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\lvr_moi\community_data\processed"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
csv_fn = "community_data.csv"
out_path = os.path.join(output_dir,  csv_fn)
community_df.to_csv(out_path, index=False, encoding='utf-8-sig')

# pkl_fn   = "預售屋備查.pkl"
# pkl_path = os.path.join(output_dir, pkl_fn)
# proc_df.to_pickle(pkl_path)